# Set up the environment

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


''

In [2]:
#Install libraries
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib
!git clone --quiet https://github.com/tensorflow/models.git

#Setting up path and others
import sys
import os
sys.path.append('/content/models/research/slim')
os.chdir('models/research')
!protoc object_detection/protos/*.proto --python_out=.

#Test the environment
#%run object_detection/builders/model_builder_test.py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-tk is already the newest version (2.7.15~rc1-1).
The following additional packages will be installed:
  libjbig0 liblcms2-2 libprotobuf10 libprotoc10 libtiff5 libwebp6
  libwebpdemux2 libwebpmux3 libxslt1.1 python-bs4 python-chardet
  python-html5lib python-olefile python-pkg-resources python-six
  python-webencodings
Suggested packages:
  liblcms2-utils python-genshi python-lxml-dbg python-lxml-doc python-pil-doc
  python-pil-dbg python-setuptools
The following NEW packages will be installed:
  libjbig0 liblcms2-2 libprotobuf10 libprotoc10 libtiff5 libwebp6
  libwebpdemux2 libwebpmux3 libxslt1.1 protobuf-compiler python-bs4
  python-chardet python-html5lib python-lxml python-olefile python-pil
  python-pkg-resources python-six python-webencodings
0 upgraded, 19 newly installed, 0 to remove and 8 not upgraded.
Need to get 3,740 kB of archives.
After this operation, 14.6 MB of addition

## Import Libraries

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import time

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from google.colab import drive

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')
 
# This is needed to display the images.
%matplotlib inline


# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [0]:
# What model to download.
# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_oid_2018_01_28'
MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_lowproposals_oid_2018_01_28'

MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'oid_bbox_trainable_label_map.pbtxt')

## Download Model
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

# Helper method
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Mount the google drive to this notebook
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# Detection

In [0]:
from PIL import Image
from io import BytesIO
import requests

#!ls /content/gdrive/'My Drive'/'Colab Notebooks'/cogs118b
filePath = '/content/gdrive/My Drive/Colab Notebooks/cogs118b/face_detection.json'
def parseData(file_path):
    with open(file_path,'r') as f:
        for data in f:
            data = data.replace("null","None")
            yield(eval(data))
TEST_IMAGE_PATHS = list(parseData(filePath))
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [0]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      #Extract the output box dimension
      #faceIndex = np.where(output_dict['detection_classes']==4)[0]
      #faceDim = output_dict['detection_boxes'][faceIndex,:]
  return output_dict

In [0]:
#Error solution when converting the image to numpy
err = 0
pred_list = []
for i, image_path in enumerate(TEST_IMAGE_PATHS):
    try:
        print("current image is ", i)
        # Prediction
        response = requests.get(image_path['content'])
        image = Image.open(BytesIO(response.content))
        image_np = load_image_into_numpy_array(image)
        image_np_expanded = np.expand_dims(image_np, axis=0)
        t1 = time.time()
        pred = run_inference_for_single_image(image_np, detection_graph)
        pred_list.append(pred)
        t2 = time.time()
        print("current time is ", t2-t1)
    except:
        print('err')
        err += 1
        pred_list.append(np.array([0,0,0,0], dtype='float32'))
        pass
print('\nDone!!!!!')
print('length is ', len(TEST_IMAGE_PATHS))
print('number of error is ', err)

current image is  0
current time is  40.37906503677368
current image is  1
err
current image is  2
current time is  38.717894315719604
current image is  3
err
current image is  4
current time is  39.14572095870972
current image is  5
current time is  37.548378467559814
current image is  6
current time is  39.146589517593384
current image is  7
current time is  36.12860441207886
current image is  8
current time is  39.073885917663574
current image is  9
current time is  38.68684148788452
current image is  10
current time is  36.85921573638916
current image is  11
current time is  36.51364541053772
current image is  12
current time is  39.59068775177002
current image is  13
current time is  38.67709255218506
current image is  14
current time is  37.56634497642517
current image is  15
current time is  39.55834341049194
current image is  16
current time is  40.35276389122009
current image is  17
current time is  37.88818144798279
current image is  18
current time is  36.81369185447693
curr

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/cogs118b/this_one.txt','w') as f:
    for np_array in pred_list:
      a_list = np_array.tolist()
      f.write("%s\n" % a_list)